<a href="https://colab.research.google.com/github/fvfarahani/GNN/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Install required packages.
# !pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
# !pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
# !pip uninstall torch-scatter
# !pip uninstall torch-sparse
# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cu116.html 
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q torch-geometric
!pip install tsfresh
!pip install karateclub
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


To activate GPU as the hardware accelerator, go to the "Runtime" menu, select "Change runtime type", and then choose "GPU" as the hardware accelerator.

In [15]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb 14 17:56:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Checking the amount of available RAM (Random Access Memory) in your Google Colab runtime environment.

In [16]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [17]:
import numpy as np
import pickle
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
cd /content/drive/MyDrive/gnn-framework

/content/drive/MyDrive/gnn-framework


In [23]:
# This is already computed, but i left the code so that you can use it if you wish
recompute_data = False
if recompute_data:

  # Load raw files and filestack
  np1 = 'source_data/fmri_ms_240_200/ts_disk1/ts/Healthycontrol.npy'
  np2 = 'source_data/fmri_ms_240_200/ts_disk2/ts/patients.npy'
  np1 = np.load(np1)
  np2 = np.load(np2)
  npa = np.vstack([np1, np2])
  try:
    os.mkdir('source_data/fmri_ms_240_200/ts')
  except FileExistsError:
      pass
  np.save('source_data/fmri_ms_240_200/ts/all.npy', np.round(npa.astype(np.float16), 3))

  np1 = 'source_data/fmri_ms_240_200/fc_disk1/fc/corr_Healthycontrol.npy'
  np2 = 'source_data/fmri_ms_240_200/fc_disk2/fc/corr_patients.npy'
  np1 = np.load(np1)
  np2 = np.load(np2)
  npa = np.vstack([np1, np2])
  try:
    os.mkdir('source_data/fmri_ms_240_200/fc')
  except FileExistsError:
      pass
  np.save('source_data/fmri_ms_240_200/fc/all.npy', np.round(npa.astype(np.float16), 3))

  # read patient id files and create target variable list (y_list)
  s1 = 'source_data/fmri_ms_240_200/subjectsID_disk1.npy'
  s2 = 'source_data/fmri_ms_240_200/subjectsID_disk2.npy'
  s1 = np.load(s1)
  s2 = np.load(s2)
  y_list = [0] * len(s1)
  y_list.extend([1] * len(s2))

  with open('source_data/fmri_ms_240_200/patienthc_y_list.pkl', 'wb') as f:
    pickle.dump(y_list, f)

else:
  with open('source_data/fmri_ms_199_200/patienthc_y_list.pkl', 'rb') as f:
    y_list = pickle.load(f) 

In [24]:
print(len(y_list))
print(y_list)

199
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [26]:
!pip uninstall numpy
!pip install numpy==1.23.5

Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.8
    /usr/local/lib/python3.8/dist-packages/numpy-1.21.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libopenblasp-r0-2d23e62b.3.17.so
    /usr/local/lib/python3.8/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.8/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.21.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 75.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you h

In [27]:
!python3 run.py

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
The analyzed config is:  config_fmri_ms_400_240_gcn.py

Now analyzing data set: fmri_ms_240_400
Now running model: GCN
Feature set extracted from time series: ts_fresh
compute node embeddings: True
node embedding method: NodeSketch
number of hidden channels:60
threshold: -2
graph type: full
batch size: 32
add_degree_to_node_features: False

Loading data set
Traceback (most recent call last):
  File "run.py", line 235, in <module>
    run_all_experiments(model_list, hidden_channel_list, threshold_list, graph_type_list,
  File "run.py", line 197, in run_all_experiments
    run(config)
  File "run.py", line 23, in run
    dataset, test_run_name_for_whole_dataset, y_list, test_run_name = gfw.utils.load_dataset(config)
  File "/content/drive/MyDr